In [ ]:
import enum
import typing
import random

import pydantic
import langchain_core
import tqdm.auto
import pandas as pd

from consol.output_formats import AbstractOutput, ReasonedMixin, FloatOutput, ABCDEFOutput
from consol.confidence_models import AbstractConfidenceModel, MsprtConfidenceModel, SprtConfidenceModel, PValueConfidenceModel, BayesianPosteriorConfidenceModel, VoteConfidenceModel

class LlmModelEnum(enum.StrEnum):
    GPT_4O = "gpt-4o"
    GPT_4O_MINI = "gpt-4o-mini"
    O3_MINI_LOW = "o3-mini-low"
    O3_MINI_MEDIUM = "o3-mini-medium"
    O3_MINI_HIGH = "o3-mini-high"
    OLLAMA_LLAMA3_1_8B = "ollama:llama3.1:8b"
    OLLAMA_QWQ_32B = "ollama:qwq:32b"
    GEMINI_2_0_FLASH = "gemini-2.0-flash"
    GEMINI_2_0_FLASH_LITE = "gemini-2.0-flash-lite"


class ConfidenceModelEnum(enum.StrEnum):
    Msprt = "msprt"
    Sprt = "sprt"
    Pvalue = "pvalue"
    BayesianPosterior = "bayesian_posterior"
    Vote40 = "vote40"
    Vote1 = "vote1"

class OutputTypeEnum(enum.StrEnum):
    Float = "float"
    Abcdef = "abcdef"

class ConfidentSolverConfig(pydantic.BaseModel):
    llm_model: LlmModelEnum

class ConfidentSolver:
    def __init__(
        self,
        confidence_model: typing.Union[ConfidenceModelEnum, AbstractConfidenceModel],
        answer_generator = typing.Generator[float, None, None],
    ):
        self.answer_generator = answer_generator

        if confidence_model == ConfidenceModelEnum.Msprt:
            self.confidence_model = MsprtConfidenceModel()
        elif confidence_model == ConfidenceModelEnum.Sprt:
            self.confidence_model = SprtConfidenceModel()
        elif confidence_model == ConfidenceModelEnum.Pvalue:
            self.confidence_model = PValueConfidenceModel()
        elif confidence_model == ConfidenceModelEnum.BayesianPosterior:
            self.confidence_model = BayesianPosteriorConfidenceModel()
        elif confidence_model == ConfidenceModelEnum.Vote40:
            self.confidence_model = VoteConfidenceModel()
        elif confidence_model == ConfidenceModelEnum.Vote1:
            self.confidence_model = VoteConfidenceModel(max_trials=1)
        elif isinstance(confidence_model, AbstractConfidenceModel):
            self.confidence_model = confidence_model
        else:
            raise ValueError(f"Unknown Confidence Model: {confidence_model}")

    def invoke(self, debug=False, **kwargs):
        max_trials = self.confidence_model.config.max_trials
        total_raw_outputs = []
        with tqdm.auto.tqdm(total=max_trials, disable=True) as pbar:
            while True:
                first, second = self._get_top_two_answers(total_raw_outputs)
                trials = self._determine_trials(first, second, max_trials, len(total_raw_outputs))
                if trials == 0:
                    pbar.close()
                    break
                raw_outputs = self._collect_raw_outputs(trials, **kwargs)
                total_raw_outputs += raw_outputs
                pbar.update(trials)
        df = self._create_dataframe(total_raw_outputs)
        if debug:
            return df
        return df['answer'].mode().iloc[0]

    def _prepare_messages(self, input):
        if isinstance(input, str):
            return [langchain_core.messages.HumanMessage(input)]
        return input

    def _get_top_two_answers(self, total_raw_outputs):
        total_ss = pd.Series([x['answer'] for x in total_raw_outputs]).value_counts()
        two = total_ss.sort_values(ascending=False).head(2).to_list()
        while len(two) < 2:
            two += [0]
        return two[0], two[1]

    def _determine_trials(self, first, second, max_trials, current_trials):
        for trials in range(0, max_trials + 1):
            if first + trials == 0:
                continue
            if self.confidence_model.test(first + trials, second):
                break
        if trials >= max_trials - current_trials:
            trials = max_trials - current_trials
        return trials

    def _collect_raw_outputs(self, trials, **kwargs):
        raw_outputs = [{
            'answer': answer,
            'token_usage': 1,
        } for answer in random.choices(self.answer_generator, k=trials)]
        return raw_outputs

    def _create_dataframe(self, total_raw_outputs):
        return pd.DataFrame({
            'answer': [x['answer'] for x in total_raw_outputs],
            'token_usage': [x['token_usage'] for x in total_raw_outputs],
        })

df = ConfidentSolver(
    answer_generator=[1,2,3],
    confidence_model="bayesian_posterior",
).invoke(debug=True)
print(df.answer.mode().iloc[0])
df